# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

&emsp;&emsp;<font face="仿宋">在案例的第二部分中，我们详细介绍了常用特征转化方法，其中有些是模型训练之必须，如自然数编码、独热编码，而有些方法则是以提高数据质量为核心、在大多数时候都是作为模型优化的备选方法，如连续变量分箱、数据标准化等。当然，在此之后，我们首先尝试构建一些可解释性较强的模型来进行用户流失预测，即采用逻辑回归和决策树模型来进行预测，并同时详细介绍了两种模型在实战中的调优技巧，在最终模型训练完成后，我们也重点讨论了关于两种可解释性模型建模结果的解释方法。

&emsp;&emsp;<font face="仿宋">从理论上来说，树模型的判别能力是要强于逻辑回归的，但在上一节最后的建模结果中我们发现两个模型的建模并无显著差别，预测准确率都维持在79%-80%之间，这或许说明很多逻辑回归无法正确判别的样本决策树模型也无法判别，据此我们推测，这是一个“入门容易、精通较难”的数据集。当然，如果我们进一步尝试其他“更强”的集成学习算法，如随机森林、XGB、CatBoost等，在当前数据集上的建模结果和逻辑回归也并无太大差异，因此我们亟需通过特征工程方法进一步提升数据集质量，进而提升最终模型效果。

&emsp;&emsp;<font face="仿宋">当然，哪怕是复杂模型在当前数据集上表现出了更好的效果，采用特征工程方法提升数据质量仍是优化建模结果必不可少的部分，正如时下流行的描述那样，“数据质量决定模型上界，而建模过程只是不断逼近这个上界”，特征工程中的一系列提高数据质量的方法、无论是在工业界实践中还是各大顶级竞赛里，都已然成了最为重要的提升模型效果的手段。

<center><img src="https://tva1.sinaimg.cn/large/008i3skNly1gwllgk4wgqj31hr0u0wh4.jpg" alt="image-20211112170651500" style="zoom:15%;" />

&emsp;&emsp;<font face="仿宋">不过，所谓的通过特征工程方法提高数据质量，看似简单但实际操作起来却并不容易。其难点并不在于其中具体操作方法的理解，至少相比机器学习算法原理，特征工程的很多方法并不复杂，特征工程的最大难点在于配合模型与数据进行方法选择、以及各种方法的工程化部署实现。一方面，特征工程方法众多，需要根据实际情况“因地制宜”，但数据的情况千变万化，很多时候需要同时结合数据探索结论、建模人员自身经验以及对各种备选方法的熟悉程度，才能快速制定行之有效的特征工程策略；另一方面，很多特征工程方法不像机器学习算法有现成的库可以直接调用，很多方法、尤其是一些围绕当前数据集的定制方法，需要自己手动实现，而这个过程就对建模人员本身的代码编写能力及工程部署能力提出了更高的要求。总而言之，特征工程是一个实践高度相关的技术，这也是为何课程会在介绍案例的过程中同步介绍特征工程常用方法的原因。

&emsp;&emsp;<font face="仿宋">当然，从宽泛的角度来看，所有围绕数据集的数据调整工作都可以看成是特征工程的一部分，包括此前介绍的缺失值填补、数据编码、特征变换等，这些方法其实都能一定程度提升数据质量，而本节开始，我们将花费一整节的时间来讨论另一类特征工程方法：特征衍生与特征筛选。而该方法通过创建更多特征来提供更多捕捉数据规律的维度，从而提升模型效果。当然特征衍生也是目前公认的最为有效的、能够显著提升数据集质量方法。

# <center>Part 1.特征衍生基础方法精讲

&emsp;&emsp;本阶开始我们将重点讨论特征工程中的特征衍生与特征筛选方法，并借此进一步提升模型效果。首先需要将此前的操作中涉及到的第三方库进行统一的导入：

In [1]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin

# 自定义模块
from telcoFunc import *

# re模块相关
import inspect, re

其中telcoFunc是自定义的模块，其内保存了此前自定义的函数和类，后续新增的函数和类也将逐步写入其中，telcoFunc.py文件随课件提供，需要将其放置于当前ipy文件同一文件夹内才能正常导入。

&emsp;&emsp;接下来导入数据并执行Part 1中的数据清洗步骤。

In [2]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [3]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

接下来即可直接带入数据进行特征衍生。

- 特征衍生基本概念与分类

&emsp;&emsp;所谓特征衍生，指的是通过既有数据进行新特征的创建，特征衍生有时也被称为特征创建、特征提取等。总体来看，特征衍生有两类方法，其一是依据数据集特征进行新特征的创建，此时的特征衍生其实是一类无监督的特征衍生，例如把月度费用（'MonthlyCharges'）和总费用（'TotalCharges'）两列相加，创建新的一列；而另外一种情况是将数据集标签情况也纳入进行考虑来创建新的特征，此时特征衍生其实是有监督的特征衍生，如上一小节中介绍的通过决策树的建模结果对连续变量进行分箱（分箱后的列也是创建的新的一列，只是有时我们会将其替换原始列）。在大多数时候特征衍生特指无监督特征衍生，而有监督的特征衍生我们会称其为目标编码。

&emsp;&emsp;而无论是特征衍生还是目标编码，实现的途径都可以分为两种，其一是通过深入的数据背景和业务背景分析，进行人工字段合成，这种方法创建的字段往往具有较强的业务背景与可解释性，同时也会更加精准、有效的提升模型效果，但缺点是效率较慢，需要人工进行分析和筛选，其二则是抛开业务背景，直接通过一些简单暴力的工程化手段批量创建特征，然后从海量特征池中挑选有用的特征带入进行建模，这种方法简单高效，但工程化方法同质化严重，在竞赛时虽是必备手段，但却难以和其他同样采用工程化手段批量创建特征的竞争者拉开差距。因此，在实际应用时，往往是先通过工程化方法批量创建特征提升模型效果，然后再围绕当前建模需求具体问题具体分析，尝试人工创建一些字段来进一步提升模型效果。

&emsp;&emsp;当然，由于我们此前已经进行了一定程度的业务背景分析和数据探索，外加考虑到代码实现难度由易到难的讲解顺序，我们将先讨论关于人工字段合成的方法，然后再介绍工程化批量创建字段的方法。

# <center>Part 3.2 批量自动化特征衍生

&emsp;&emsp;在介绍完手动特征衍生方法后，接下来我们来讨论如何通过一些自动化的方法、批量创建海量特征。在正式讨论这部分内容之前，我们需要回顾和总结此前手动特征衍生过程中的部分要点，通过对这些内容的回顾，我们能快速构建对批量特征衍生方法的整体认知：

- 特征衍生的本质

&emsp;&emsp;所谓特征衍生，其本质指的是对既有数据信息的重新排布。需要知道的是，特征衍生本生并不是去创造更多的信息，而仅仅是借助现有的数据去组合出一些新的数据，其本质属于信息重排。当然，尽管只是信息重排，但对于建模结果的提升效果却是显而易见的。

- 特征衍生的过程

&emsp;&emsp;通过上面手动特征衍生的过程，不难发现，特征衍生的“信息重排”的过程就是简单的围绕单个列进行变换或者围绕多个列进行组合。例如，new_customer字段的创建，其本质上就是围绕tenure字段进行的变换，即把所有tenure取值为1的用户都标记为1，其他用户标记为0，当然，如果我们更进一步来进行思考，这其实就是tenure字段进行独热编码后的某一列；再比如，service_num字段其本质就是所有记录各项服务购买情况的字段（先转化为数值型变量再）进行求和汇总的结果，即多列进行组合变换。

- 手动特征衍生与批量特征衍生

&emsp;&emsp;当然，无论是从特征衍生的本质本质、还是从特征衍生的过程上来看，接下来将要介绍的批量自动化特征衍生，并没有超出上述介绍的范畴，即批量特征衍生同样也是数据信息的重排过程，并且执行过程上，同样也是借助单独的列进行变换或者是多个列进行组合变换。只不过不同的是，批量特征衍生并不会像手动特征衍生一样，先从思路出发、再分析数据集当前的业务背景或数据分布规律、最后再进行特征衍生，而是优先考虑从方法出发，直接考虑单个列、不同的列之间有哪些可以衍生出新特征的方法，然后尽可能去衍生出更多的特征。

&emsp;&emsp;例如，我们只需要对tenure字段进行独热编码就可以一次性衍生出73个列，甚至，我们还可以随机选取tenure字段的两个不同值（例如tenure=15、17），并把这两个时间点入网的用户标记为1、其他用户标记为0，而在随机选取的情况下，这里就有$C^2_{73}=\frac{73*72}{2}=2628$种特征衍生的方法。

- 特征衍生、海量特征与特征筛选

&emsp;&emsp;关于有哪些可以用于特征衍生的方法，我们稍后会详细介绍。但这里需要注意的是，在这种尽可能衍生更多特征的的基本指导思想下，自动化批量特征衍生往往会创造出非常多的特征，而这些特征并不是每个都能帮助模型训练的出更好的结果，并且特征列本身过多也会极大程度上影响建模效率。例如，继续上面的例子，我们只需要对tenure字段进行独热编码就可以一次性衍生出73个列，而在随机选取tenure字段的两个不同值（例如tenure=15、17组合出的$C^2_{73}=\frac{73*72}{2}=2628$种特征中，大多数特征也属于无用特征。

&emsp;&emsp;因此若要进行自动批量特征衍生，往往是一定需要搭配特征筛选方法的，也就是需要借助一些策略，来对批量创建的海量特征进行筛选，“去粗取精”，选出最能提升模型效果的特征，在提高模型效果的同时提升建模效率。例如此前的IV值筛选特征就是一种特征筛选的方法，当然本小节我们先介绍关于自动化批量特征衍生的方法，下一小节我们再重点介绍更多特征筛选策略。

<center><img src="https://s2.loli.net/2022/01/22/jyYBXTRvHSNluIJ.png" alt="image-20220122122112626" style="zoom:33%;" />

&emsp;&emsp;此外，批量特征衍生还将造成另外的一个问题，那就是很多衍生出来的特征并不具备业务层面的可解释性，例如tenure=15、17组合衍生出来的特征，我们就无法从业务的角度解释为何需要这么做，但是，这并不代表该特征就一定无法帮助模型训练得出一个更好的结果。在机器学习整体都是后验思想为主导的情况下，我们往往不会过于关注批量衍生的特征具体的业务含义，而只考虑最终的建模效果，也就是说，批量特征衍生+特征筛选的策略，完全是一个“依据建模结果说话”的策略。

- 特征衍生方法汇总

&emsp;&emsp;接下来我们将首先全面详细的介绍常用特征衍生方法，并重点讨论相关方法的实现代码，这里我们先考虑如何把特征“做多”，然后再考虑如何把特征“做精”。总的来说，批量特征衍生有如下方法划分：

<center><img src="https://s2.loli.net/2022/01/22/gANubs8OwQ256Dd.png" alt="特征衍生方法" style="zoom:33%;" />

- 本节学习过程注意事项

&emsp;&emsp;相比有严谨理论体系的机器学习算法，特征衍生的相关方法更像是人们在长期实践过程中总结出来的方法论，这些方法切实有效，但（暂时）却没有一套能够完整统一的理论体系来“框住”这些方法。因此特征衍生的方法很多时候都是“就方法讨论方法”，很多方法使用的技巧和注意事项也都是经验之谈。此外，由于特征衍生应用场景复杂多变，需要综合数据体量、数据规律、现有算力等因素进行考虑，截至目前也并没有统一的第三方库能够提供完整特征衍生的方法实现，因此本节每个特征衍生的方法介绍，我们都将从三个角度入手来进行讨论，分别是该方法的执行过程、使用时的注意事项以及实现代码。尤其需要注意的是，本节会提供所有常用特征衍生方法封装的函数，这些函数或简单或复杂，但都将被后续案例课长期反复复用，当然也可作为实际工作或参与竞赛时的有力工具，因此需要在学习时额外注重对本节代码的学习与掌握。

## 一、单变量特征衍生方法

&emsp;&emsp;正如此前在手动特征衍生过程中看到的那样，一般来说，我们可以借助单个变量进行特征衍生，也可以组合多个变量进行特征衍生，我们先看相对更简单的单变量特征衍生方法。

### 1.数据重编码特征衍生

&emsp;&emsp;首先需要知道的是，此前我们所介绍的所有数据重编码的过程，新创建的列都可以作为一个额外的独立特征，即我们在实际建模过程中，不一定是用重编码后新的列替换掉原始列，而是考虑同时保留新的特征和旧的特征，带入到下一个环节、即特征筛选来进行特征筛选，如果数据重编码后的特征是有效的，则自然会被保留，否则则会被剔除。

&emsp;&emsp;这么一来，我们或许就不用考虑在当前模型下是否需要进行数据重编码，而是无论是否需要，都先进行重编码、并同时保留原特征和衍生出来的新的特征。此处我们简单回顾此前所介绍的一系列数据重编码的方法：

- 连续变量数据重编码方法      
    - 标准化：0-1标准化/Z-Score标准化
    - 离散化：等距分箱/等频分箱/聚类分箱

- 离散变量数据重编码方法
    - 自然数编码/字典编码
    - 独热编码/哑变量变换

&emsp;&emsp;当然，在同时保留原始列和重编码的列时，极有可能出现原始列和重编码的列都是有效特征的情况，例如此前我们看到的tenure独热编码后的某列（tenure=1）和原始列同时带入模型的情况。

### 2.高阶多项式特征衍生

&emsp;&emsp;对于单独的变量来说，除了可以通过重编码进行特征衍生外，还可以通过多项式进行特征衍生，即创建一些自身数据的二次方、三次方数据等。该方法我们曾在逻辑回归一节中重点介绍过，此处进行简单回顾。

- 多项式处理结果

&emsp;&emsp;假设$X1$是某原始特征，单独特征进行高阶多项式衍生过程如下：

<center><img src="https://s2.loli.net/2022/01/19/t37jW9xZ5kqUeGd.png" alt="image-20220119174222374" style="zoom:33%;" />

- 多项式衍生实现方法

&emsp;&emsp;上述过程较为简单，直接利用数组广播特性，手动实现过程也较为简单。当然我们更推荐使用sklearn中的PolynomialFeatures评估器来实现该过程。该评估器不仅能够非常便捷的实现单变量的多项式衍生，也能够快速实现多变量组合多项式衍生，且能够与机器学习流集成，也便于后续的超参数搜索方法的使用。

In [4]:
from sklearn.preprocessing import PolynomialFeatures

In [5]:
x1 = np.array([1, 2, 4, 1, 3])

In [6]:
PolynomialFeatures(degree=5).fit_transform(x1.reshape(-1, 1))

array([[1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00],
       [1.000e+00, 2.000e+00, 4.000e+00, 8.000e+00, 1.600e+01, 3.200e+01],
       [1.000e+00, 4.000e+00, 1.600e+01, 6.400e+01, 2.560e+02, 1.024e+03],
       [1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.000e+00],
       [1.000e+00, 3.000e+00, 9.000e+00, 2.700e+01, 8.100e+01, 2.430e+02]])

尽管过程简单，但多项式特征衍生也是极为常见且效果出色的特征衍生方法，在此前的逻辑回归建模实验中，简单的多项式衍生，就能够将逻辑回归的决策边界由线性改善至不规则边界，从而极大提升模型建模效果：

<center><img src="https://s2.loli.net/2022/01/19/mUEtDxkquaMWh1w.png" alt="image-20220119180207156" style="zoom:33%;" />

当然，一般来说单特征的多项式往往是针对连续变量会更有效，但再某些情况下也可以将其用于离散型随机变量。

### 3.特征衍生准则

- 无限特征

&emsp;&emsp;在上述过程的基础上，我们将这些方法稍作组合、或者是进行深入拓展，则会发现，就单独一个特征而言，我们都可以衍生出无限个特征。例如我们可以将某连续变量先进行N阶多项式衍生，然后再进行分箱，然后再进行独热编码；或者就是单纯的进行非常高阶的多项式衍生；再或者，我们看随机修改下归一化的规则（以0-1标准化为例），不再是减去最小值除以极值，而是减去次小的值、或者第三小的值等等。你会发现，哪怕是单独针对某个变量，我们都可以衍生出近乎无穷个特征。

&emsp;&emsp;但需要知道的是，尽管特征可以无限衍生，但因为算力有限、时间有限，我们不可能进行无止尽的尝试。因此，在实际模型训练过程中，也并非无节制的朝向无限特征的方向进行特征衍生，往往我们需要有些判断，即哪些情况下朝什么方向进行特征衍生是最有效的。当然，同时我们需要知道的是，特征衍生的方法极少有理论依据、或者只有零散的理论依据，例如多项式衍生其实是借助了核函数的思想，但就特征衍生的技术整体而言，并没有一个完整系统的理论体系，作为何时选择何种方法的参考依据。因此不难看出，特征衍生其实是一个极其考验建模工作者的建模经验、数据敏感度甚至是建模灵感的工作事项，而在很多实际工作和竞赛中，我们也确实会发现特征工程方法选择的不同，往往是建模结果拉开差距的关键。

> 随着课程的深入，我们还会发现，无限衍生特征的方向会有非常多，并且我们也会发现，随着特征衍生进行的深入，新特征的有效性也是在快速递减的。

&emsp;&emsp;而在茫茫多特征衍生方向中如何选择当前数据集特征衍生的方向，首先，课上会在每套特征衍生方法的后面附上这些方法的选择依据，这些依据大都源自实践经验，可以普遍适用于一般情况，并作为特征衍生的基本依据；其次，在后续的案例课中，我们也将结合具体的数据来讨论不同情况下最适用的特征衍生方法，在实践中快速积累特征衍生方法的使用经验。

- 特征衍生选择依据

&emsp;&emsp;这里我们先给出上述单变量特征衍生方法在使用过程中的选取依据，此处我们假设实际构建的模型以集成学习为主：        
- 优先考虑分类变量的独热编码，并同时保留原始变量与独热编码衍生后的变量。独热编码能够丰富树模型生长过程中备选的数据集切分点，因此能够进一步丰富集成学习中不同树模型可能的差异性。但同时也需要注意的是有两种情况不适用于使用独热编码，其一是分类变量取值水平较多（例如超过10个取值），此时独热编码会造成特征矩阵过于稀疏，从而影响最终建模效果；其二则是如果该离散变量参与后续多变量的交叉衍生（稍后会介绍），则一般需再对单独单个变量进行独热编码；      
- 优先考虑连续变量的数据归一化，尽管归一化不会改变数据集分布，即无法通过形式上的变换增加树生长的多样性，但归一化能够加快梯度下降的执行速度，加快迭代收敛的过程；
- 在连续变量较多的情况下，可以考虑对连续变量进行分箱，原因同第一点。具体分享方法优先考虑聚类分箱，若数据量过大，可以使用MiniBatch K-Means提高效率，或者也可以简化为等频率/等宽分箱；      
- 不建议对单变量使用多项式衍生方法，相比单变量的多项式衍生，带有交叉项的多变量的多项式衍生往往效果会更好。

## 二、双变量特征衍生方法

&emsp;&emsp;接下来我们进一步介绍涉及多个变量的特征衍生方法。在大多数情况下，多个变量的交叉组合往往都比单变量特征衍生更有价值。而该过程我们在手动特征衍生时也见到了多次，例如我们创建的老年且经济不独立的标识字段、按月付费且无纸质合约类账户等，都是两个变量的交叉组合结果；而每位用户购买服务总数字段，则更是十个购买服务记录的字段求和之后的结果。一般来说如果具体细分的化，两个特征组合成新的字段我们会称其为双变量（或者双特征）交叉衍生，而如果涉及到多个字段组合，则会称其为多变量交叉衍生。一般来说，双变量特征衍生是目前常见特征衍生方法中最常见、同样也是效果最好的一类方法，这也是我们接下来要重点介绍的方法。而多变量特征衍生，除了四则运算（尤其以加法居多）的组合方法外，其他衍生方法随着组合的字段增加往往会伴随非常严重的信息衰减，因此该类方法除特定场合外一般不会优先考虑使用。我们会在介绍完双变量衍生方法后再介绍多变量衍生。

&emsp;&emsp;接下来开始介绍双变量特征衍生方法，本部分我们会以各个不同的方法作为主线进行介绍，并在每个方法后介绍该方法的使用场景。

### 1.四则运算特征衍生

&emsp;&emsp;先从简单的方法开始讨论——四则运算特征衍生。该过程非常简单，就是单纯的选取两列进行四则运算，基本过程如下：

<center><img src="https://s2.loli.net/2022/01/20/hZOarqcndsHDxlV.png" alt="image-20220120123540522" style="zoom:33%;" />

该过程并不复杂，实际代码执行过程也只需要单独索引出两列然后进行四则运算即可。一般来说，四则运算特征衍生的使用场景较为固定，主要有以下三个：     
- 其一是用于创建业务补充字段：在某些数据集中，我们需要通过四则运算来创建具有明显业务含义的补充字段，例如在上述电信用户流失数据集中，我们可以通过将总消费金额除以用户入网时间，即可算出用户平均每月的消费金额，或者使用用户每月消费金额除以购买服务总数，则可算出每项服务的平均价格，这些字段有明确的业务含义，我们甚至可以将其视作原始字段；          
- 其二，往往在特征衍生的所有工作结束后，我们会就这一系列衍生出来的新字段进行四则运算特征衍生，作为数据信息的一种补充；      
- 其三，在某些极为特殊的字段创建过程中使用，例如竞赛中常用的黄金组合特征、流量平滑特征（稍后会重点讨论）等，需要使用四则运算进行特征衍生。

### 2.交叉组合特征衍生

- 方法介绍

&emsp;&emsp;所谓交叉组合特征衍生，指的是不同分类变量不同取值水平之间进行交叉组合，从而创建新字段的过程。例如此前我们创建的老年且经济不独立的标识字段，实际上就是是否是老年人字段（SeniorCitizen）和是否经济独立字段（Dependents）两个字段交叉组合衍生过程中的一个：

<center><img src="https://s2.loli.net/2022/01/20/BiH4LtVTOjkWQuI.png" alt="image-20220120140301256" style="zoom:33%;" />

不难看出，该计算流程并不复杂，需要注意的是，交叉组合后衍生的特征个数是参数交叉组合的特征的取值水平之积，因此交叉组合特征衍生一般只适用于取值水平较少的分类变量之间进行，若是分类变量或者取值水平较多的离散变量彼此之间进行交叉组合，则会导致衍生特征矩阵过于稀疏，从而无法为模型提供有效的训练信息。

- 手动实现

&emsp;&emsp;我们仍然以telco数据集为例，尝试围绕'SeniorCitizen'、'Partner'、'Dependents'字段进行两两交叉组合衍生，当然该流程也可以顺利推广至任意多个任意取值个数的分类变量两两交叉组合衍生过程。

In [7]:
# 数据集中离散变量
category_cols

['gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod']

In [8]:
# 提取目标字段
colNames = ['SeniorCitizen', 'Partner', 'Dependents']

In [9]:
# 单独提取目标字段的数据集
features_temp = features[colNames]
features_temp.head(5)

,SeniorCitizen,Partner,Dependents
0,0,Yes,No
1,0,No,No
2,0,No,No
3,0,No,No
4,0,No,No


In [10]:
# 创建空列表用于存储衍生后的特征名称和特征
colNames_new_l = []
features_new_l = []

In [11]:
# enumerate过程
for col_index, col_name in enumerate(colNames):
    print(col_index, col_name)

0 SeniorCitizen
1 Partner
2 Dependents


In [12]:
# 衍生特征列名称
for col_index, col_name in enumerate(colNames):
    for col_sub_index in range(col_index+1, len(colNames)):
        newNames = col_name + '&' + colNames[col_sub_index]
        print(newNames)

SeniorCitizen&Partner
SeniorCitizen&Dependents
Partner&Dependents


In [13]:
# 创建衍生特征列名称及特征本身
for col_index, col_name in enumerate(colNames):
    for col_sub_index in range(col_index+1, len(colNames)):
        
        newNames = col_name + '&' + colNames[col_sub_index]
        colNames_new_l.append(newNames)
        
        newDF = pd.Series(features[col_name].astype('str') 
                          + '&'
                          + features[colNames[col_sub_index]].astype('str'), 
                          name=col_name)
        features_new_l.append(newDF)

In [14]:
features_new = pd.concat(features_new_l, axis=1)
features_new.columns = colNames_new_l

In [15]:
features_new

,SeniorCitizen&Partner,SeniorCitizen&Dependents,Partner&Dependents
0,0&Yes,0&No,Yes&No
1,0&No,0&No,No&No
2,0&No,0&No,No&No
3,0&No,0&No,No&No
4,0&No,0&No,No&No
...,...,...,...
7038,0&Yes,0&Yes,Yes&Yes
7039,0&Yes,0&Yes,Yes&Yes
7040,0&Yes,0&Yes,Yes&Yes
7041,1&Yes,1&No,Yes&No


In [16]:
colNames_new_l

['SeniorCitizen&Partner', 'SeniorCitizen&Dependents', 'Partner&Dependents']

截至目前，我们创建了3个4分类的变量，我们可以直接将其带入进行建模，但需要知道的是这些四分类变量并不是有序变量，因此往往我们需要进一步将这些衍生的变量进行独热编码，然后再带入模型：

In [17]:
enc = preprocessing.OneHotEncoder()

In [18]:
enc.fit_transform(features_new)

<7043x12 sparse matrix of type '<class 'numpy.float64'>'
	with 21129 stored elements in Compressed Sparse Row format>

In [19]:
# 借助此前定义的列名称提取器进行列名称提取
cate_colName(enc, colNames_new_l, drop=None)

['SeniorCitizen&Partner_0&No',
 'SeniorCitizen&Partner_0&Yes',
 'SeniorCitizen&Partner_1&No',
 'SeniorCitizen&Partner_1&Yes',
 'SeniorCitizen&Dependents_0&No',
 'SeniorCitizen&Dependents_0&Yes',
 'SeniorCitizen&Dependents_1&No',
 'SeniorCitizen&Dependents_1&Yes',
 'Partner&Dependents_No&No',
 'Partner&Dependents_No&Yes',
 'Partner&Dependents_Yes&No',
 'Partner&Dependents_Yes&Yes']

In [20]:
# 最后创建一个完整的衍生后的特征矩阵
features_new_af = pd.DataFrame(enc.fit_transform(features_new).toarray(), 
                               columns = cate_colName(enc, colNames_new_l, drop=None))

In [21]:
features_new_af.head(5)

,SeniorCitizen&Partner_0&No,SeniorCitizen&Partner_0&Yes,SeniorCitizen&Partner_1&No,SeniorCitizen&Partner_1&Yes,SeniorCitizen&Dependents_0&No,SeniorCitizen&Dependents_0&Yes,SeniorCitizen&Dependents_1&No,SeniorCitizen&Dependents_1&Yes,Partner&Dependents_No&No,Partner&Dependents_No&Yes,Partner&Dependents_Yes&No,Partner&Dependents_Yes&Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### 3.多变量的多项式特征衍生

- 方法介绍

&emsp;&emsp;接下来进一步介绍多变量的多项式衍生，多变量的多项式衍生过程并不复杂，无非就是在多项式计算时带入更多特征进行计算，而在进行多项式计算时并无本质区别，都是进行幂运算和交叉项计算。首先回顾双变量二阶多项式衍生过程：

<center><img src="https://s2.loli.net/2022/01/20/MKXbm2tpf1hFVRw.png" alt="image-20220120222202519" style="zoom:33%;" />

此时如果我们增加参与多项式计算的特征，则有如下多变量多项式衍生过程：

<center><img src="https://s2.loli.net/2022/02/13/2Hunyt6rRGeKf1T.png" alt="image-20220213193758668" style="zoom:33%;" />

当然，对于二阶多项式衍生，多变量的计算过程其实等价于多个双变量组合衍生的结果，例如上述X1、X2、X3三个变量的二阶多项式衍生，其实就等价于X1和X2、X1和X3（或X2、X3）两组二阶多项式衍生后去重的结果，因此如果是二阶多项式衍生的话，多变量的交叉组合结果完全等价于多变量中两两变量的二阶多项式衍生结果。但如果是三阶甚至是更高阶的多项式衍生，结果就会有所不同：

<center><img src="https://s2.loli.net/2022/02/13/Ser2R1dj975XlZC.png" alt="image-20220213114830610" style="zoom:33%;" />

<center><img src="https://s2.loli.net/2022/02/13/mMhWDJAXUiPeubf.png" alt="image-20220213114835000" style="zoom:33%;" />

不难发现，多变量的多项式衍生和多变量两两交叉多项式衍生的差异主要体现在多个变量的交叉项上，当然伴随着多项式阶数增加，多变量交叉项本身也会更多更复杂，二者差异也会更加明显。当然，多个特征的交叉组合乘积也同样会增加特征的表现，但同时也会增加伴随着多变量多项式衍生的变量数量增加以及阶数的增加，特征数量也会呈指数级增加趋势，并且衍生特征的取值也将变得非常不稳定，会伴随着阶数增加绝对值快速趋近于0或者一个非常大的数。因此，和此前的多变量特征衍生方法的使用场景类似，一般是针对人工判断的非常重要的特征可以考虑进行多变量的三阶甚至四阶多项式衍生。